In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load Model

In [ ]:
configFile = 'config-mobilenet v3\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'ssd_mobilenet_v3_large_coco_2020_01_14\\ssd_mobilenet_v3_large_coco_2020_01_14\\frozen_inference_graph.pb'

In [ ]:
model = cv2.dnn_DetectionModel(frozen_model, configFile)

In [ ]:
classList = []
Labels = 'Labels.txt'
with open(Labels) as input:
    classList = input.read().splitlines()

In [ ]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean(127.5)
model.setInputSwapRB(True)

# Load Image

In [ ]:
img = cv2.imread('Images/photo2.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
imgSize = tuple(img.shape[1::-1])
imgReSize = (int(imgSize[0]/5), int(imgSize[1]/5))

# Object Detection On Image

In [ ]:
classIndex, confidence, box = model.detect(img, confThreshold=0.5)

In [ ]:
fontSize = 10
font = cv2.FONT_HERSHEY_PLAIN

for ClassItem, conf, classBox in zip(classIndex, confidence, box):
    if ClassItem > 80:
        continue
    print("Class: " + str(classList[ClassItem- 1]) + ", Confidence: " + str(round(conf * 100, 2)) + "%")
    cv2.rectangle(img, classBox, (0,0,0), 10)
    cv2.putText(img, classList[ClassItem- 1], (classBox[0]+10, classBox[1]+100), font, fontSize, (0,0,255), thickness=10)

img = cv2.resize(img, imgReSize)
cv2.imshow("Objects: ",img)
cv2.waitKey(0) 
cv2.destroyAllWindows() 


# Object Detection On Video 

In [ ]:
video = cv2.VideoCapture('Videos/People walking in the city.mp4')

# validate file
if not video.isOpened():
    raise IOError('Cannot open file')

fontSize = 2
font = cv2.FONT_HERSHEY_PLAIN

while True:
    _, frame = video.read()

    classIndex, confidence, box = model.detect(frame, confThreshold=0.5)

    if len(classIndex) != 0:
        for ClassItem, conf, classBox in zip(classIndex, confidence, box):
            if ClassItem > 80:
                continue
            print("Class: " + str(classList[ClassItem- 1]) + ", Confidence: " + str(round(conf * 100, 2)) + "%")
            cv2.rectangle(frame, classBox, (0,0,0), 2)
            cv2.putText(frame, classList[ClassItem- 1], (classBox[0]+10, classBox[1]+40), font, fontSize, (0,0,255), thickness=2)

    cv2.imshow("Objects: ", frame)

    print('frame')

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
    
video.release()
cv2.destroyAllWindows() 
